In [1]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os 
from datetime import datetime


In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "4g").getOrCreate()

In [3]:
#đọc data
def read_data(path):
    df = spark.read.json(path)
    return df

#Xử lý data
def tranfrom_data(df):
    df = df.select('_source.*')
    df = df.drop('Mac')
    df.select('AppName').distinct()
    df = df.withColumn("Category",
        when((col("AppName") == 'CHANNEL') | 
            (col("AppName") =='KPLUS'), "Truyền Hình")
        .when((col("AppName") == 'VOD') | 
            (col("AppName") =='FIMS'), "Phim Truyện")
        .when((col("AppName") == 'RELAX'), "Giải Trí")
        .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
        .when((col("AppName") == 'SPORT'), "Thể Thao")
        .otherwise("Error"))
    df = df.select('Contract','Category','TotalDuration')
    return df

#tổng hợp và pivot
def summarize_and_pivot_data(df):
    df = df.groupBy('Contract','Category').sum('TotalDuration').withColumnRenamed('sum(TotalDuration)','TotalDuration')
    result =  df.groupBy('Contract').pivot('Category').sum('TotalDuration').fillna(0)
    return result

def convert_to_datevalue(value):
    date_value = datetime.strptime(value, "%Y%m%d").date()
    return date_value

def date_range(start_date, end_date):
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime("%Y%m%d"))  # Trả về định dạng chuỗi YYYYMMDD
        current_date += timedelta(days=1)  # Sử dụng timedelta
    return date_list

def generate_date_range(from_date, to_date):
    from_date = convert_to_datevalue(from_date)
    to_date = convert_to_datevalue(to_date)
    date_list = date_range(from_date, to_date)
    return date_list

#chuyển tên file thành ngày VD: 20220104.json thành 2022-01-04
def convert_filename_to_date(filename):
    # Lấy phần chuỗi chứa ngày từ tên file
    date_str = filename.split('.')[0]  # Lấy chuỗi trước dấu chấm
    # Chuyển đổi định dạng từ YYYYMMDD thành YYYY-MM-DD
    date_obj = datetime.strptime(date_str, '%Y%m%d').date()  # Chuyển đổi thành đối tượng datetime
    return date_obj

#hàm đọc nhiều file và tổng hợp thành 1 files
def read_listfile(list_file,path):
    final_result = None
    for file in list_file:
        print(f"==> Reading file {file} <==")
        path_new = path + '\\' + file + '.json'

        data_now = read_data(path_new) #đọc
        data_trans = tranfrom_data(data_now) #tranform
        data_sum = summarize_and_pivot_data(data_trans) #summarize
         
        data_sum.show()
        
        dated = convert_filename_to_date(file)
        data_now = data_sum.withColumn("date", lit(dated))  # Thêm cột 'date'

    if final_result is None:
        final_result = data_now  # Gán giá trị lần đầu
    else:
        final_result = final_result.union(data_now)  # Nối với DataFrame hiện tại
    return final_result

def process_range_days(df):
    print('------------------------')
    print('Starting merce data range days')
    result = df.groupBy('Contract') \
           .sum('Giải Trí', 'Phim Truyện', 'Thiếu Nhi', 'Thể Thao', 'Truyền Hình') \
           .withColumnRenamed('sum(Giải Trí)', 'Total_Giải_Trí') \
           .withColumnRenamed('sum(Phim Truyện)', 'Total_Phimm_Truyện') \
           .withColumnRenamed('sum(Thiếu Nhi)', 'Total_Thiếu_Nhi') \
           .withColumnRenamed('sum(Thể Thao)', 'Total_Thể_Thao') \
           .withColumnRenamed('sum(Truyền Hình)', 'Total_Truyền_Hình')
    return result

In [4]:


path = 'D:\\Hoc_DE\\Data\\Dataset\\log_content'
list_file = generate_date_range('20220401', '20220403')
count_list = len(list_file)

print(f"==> Starting read {count_list} file <==")
print('------------------------')
df = read_listfile(list_file, path)

final_result = process_range_days(df)
final_result.show()


NameError: name 'timedelta' is not defined